<a href="https://colab.research.google.com/github/lakshitgosain/Contextual-Chatbot/blob/main/Contextual_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk 
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import tensorflow as tf
import numpy as np
import tflearn
import random 
import json

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
pip install tflearn

     |████████████████████████████████| 112kB 5.6MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=5f02bac121d1c380d5e3ac5aeb4ad95e898f8b785223afc8f789c3eebf2217d6
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [ ]:
from google.colab import files


In [ ]:
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day"],\r\n         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["Bye", "See you later", "Goodbye"],\r\n         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\r\n        },\r\n        {"tag": "thanks",\r\n         "patterns": ["Thanks", "Thank you", "That\'s helpful"],\r\n         "responses": ["Happy to help!", "Any time!", "My pleasure"]\r\n        },\r\n        {"tag": "hours",\r\n         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],\r\n         "responses": ["We\'re open every day 9am-9pm", "Our hours are 9am-9pm every day"]\r\n        },\r\n        {"tag": "location",\r\n         "patterns": 

In [ ]:
#importing the chatbot intent file.
with open('intents.json') as json_data:
  intents=json.load(json_data)

In [ ]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [ ]:
words=[]#for stroring individual words
classes=[]#Storing tags
documents=[]#Storing documents
ignore=['?']#Ignore words and characters we do not want to consider

#loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the sentence
    w=nltk.word_tokenize(pattern)
    #add word to the words list
    words.extend(w)
    #adding words to the documents
    documents.append((w,intent['tag']))
    #add tags to our class list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
words=[stemmer.stem(w.lower()) for w in words if w not in ignore]
words=sorted(list(set(words)))

In [ ]:
classes=sorted(list(set(classes)))

In [ ]:
print(len(documents),documents)
print(len(classes),classes)
print(len(words),words)

31 [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['What', 'is', 'your', 'location', '?'], 'location'), (['Where', 'are', 'you', 'located', '?'], 'location'), (['What', 'is', 'your', 'address', '?'], 'location'), (['Where', 'is', 'your', 'restaurant', 'situated', '?'], 'location'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['What', 'is', 'your', 'menu', 'for', 'today', '?'], 'todaysmenu'), (['What', 'are', 'you', 'ser

In [ ]:
#Creating Training Data
training=[]
output=[]
#Create and empty array for output
output_empty=[0]*len(classes)


In [ ]:
for doc in documents:
  #initialize bag of words
  bag=[]
  pattern_words=doc[0]
  #stemming each word
  pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]

  #to create BOW arrays
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  
  output_row=list(output_empty)
  output_row[classes.index(doc[1])]=1

  training.append([bag,output_row])

#shuffling features and turning it into np.array
random.shuffle(training)
training=np.array(training)

#Creating training lists
train_x=list(training[:,0])
train_y=list(training[:,1])



In [ ]:
train_y

[[0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0]]

In [ ]:
#Resetting the underlying graph
tf.compat.v1.reset_default_graph()

#Building the neural network
net=tflearn.input_data(shape=[None,len(train_x[0])])#input layer
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,10)
net=tflearn.fully_connected(net,len(train_y[0]),activation='softmax')#no. of classes
net=tflearn.regression(net)#It is a regression layer, Used to apply regression on the provided inputs.


#Defining model and setting up the tensorboard
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

#Starting the training of the model
model.fit(train_x,train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.22523 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.22523 - acc: 0.9801 -- iter: 24/31
Training Step: 4000  | total loss: 0.20530 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.20530 - acc: 0.9821 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  #Stemming each word
  sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

#returning bag of words array: 0 or 1 for each word in the bag that exists
def bow(sentence, words, show_details=False):
  #tokenizing the pattern
  sentence_words=clean_up_sentence(sentence)
  #Generate Bag of words
  bag=[0]*len(words)
  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
        if show_details:
          print("Found in bag: %s" %w)
  
  return(np.array(bag))
                     

In [ ]:
Error_threshold=0.30#filter out predictions below the probability value
def classify(sentence):#classify each sentence and generate a corresponding probability
  #generate probabilities from the model
  results=model.predict([bow(sentence,words)])[0]
  #Filter out the predictions below the threshold
  results=[[i,r] for i,r in enumerate(results) if r>Error_threshold]
  #Sort by the strnegth of Probabilities
  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in results:
    return_list.append((classes[r[0]],r[1]))#created a tupple
  
  #Return the tuple of intent and probability
  return return_list



In [ ]:
def response(sentence,userId='123',show_details=False):
  results=classify(sentence)
  #if we have a classification then find the matching intent tag
  if results:
    #loop as long as there are matches to process
    while results:
      for i in intents['intents']:
        #find a tag maching the first results
        if i['tag']==results[0][0]:

          #a random response from the intent
          return print(random.choice(i['responses']))
      
      results.pop(0)
    

In [ ]:
classify("What are your hours of operations?")

[('hours', 0.9951466)]

In [ ]:
response("what are your hours of operations?")

Our hours are 9am-9pm every day


In [ ]:
classify("what is your menu today")

[('menu', 0.7652827)]

In [ ]:
response("what is your menu today")

You can check various delicacies given in the food menu at www.mymenu.com


In [ ]:
response("Do you accept Cash?")

We accept most major credit cards
